# Project 1, Part 3, Meal Related Queries



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [ ]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [ ]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [ ]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.3.1 How many meals were purchased for all of AGM?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item. 

The sum of quantity in the line_items table will tell you the total meals purchased.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return only 1 row into a Pandas dataframe and should look similar to this: 

||total_meals_purchased|
|---|---|
|0|8228284|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.3.2 How many meals were purchased for all of AGM by meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item. 

For meal_name, use the description column in the products table.

Sort by meal_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 8 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||meal_name|total_meals_purchased|
|---|---|---|
|0|Brocolli Stir Fry|913984|
|...|...|...|
|7|Tilapia Piccata|687237|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.3.3 How many meals were purchased by store and by meal?

For store_name use the store's city.

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

For meal_name, use the description column in the products table.

Sort by store_name in alphabetical order, then by meal_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.



The query should return 40 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|meal_name|total_meals_purchased|
|---|---|---|---|
|0|Berkeley|Brocolli Stir Fry|232038|
|...|...|...|...|
|39|Seattle|Tilapia Piccata|153448|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.3.4 How many meals were purchased by month?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the month_number (1 = January) and the month from the sale_date.

Sort by month_number.


Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.



The query should return 12 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|total_meals_purchased|
|---|---|---|---|
|0|1|January  |650319|
|...|...|...|...|
|11|12|December |695035|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.3.5 How many meals were purchased by month and meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the month_number (1 = January) and the month from the sale_date.

For meal_name, use the description column in the products table.

Sort by month_number, then by meal_name in alphabetical order.


Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**


The query should return 96 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|meal_name|total_meals_purchased|
|---|---|---|---|---|
|0|1|January  |Brocolli Stir Fry|72161|
|...|...|...|...|...|
|95|12|December |Tilapia Piccata|58260|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.3.6 How many meals were purchased by day of week and meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the dow (0 = Sunday) and the day_of_week from the sale_date.

For meal_name, use the description column in the products table.

Sort by dow, then by meal_name in alphabetical order

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 56 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||dow|day_of_week|meal_name|total_meals_purchased|
|---|---|---|---|---|
|0|0|Sunday   |Brocolli Stir Fry|172250|
|...|...|...|...|...|
|55|6|Saturday |Tilapia Piccata|135327|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df